### Query FTP

##### jupyter nbconvert --to script Continue-Update-Process.ipynb

In [1]:
import ftplib
import os
import time

# FTP server details
ftp_server = "products.impact.com"
ftp_user = "ps-ftp_5567077"
ftp_password = "6r%]mobnH6"
ftp_directory = "/Fanatics-(Global)/"
file_to_download = "Fanatics-Product-Catalog_IR.txt.gz"  # Specify the file to download

# Local directory to save the file
local_directory = "data-update-process"
os.makedirs(local_directory, exist_ok=True)  # Create the directory if it doesn't exist

max_retries = 3
retry_delay = 5  # Delay in seconds between retries

def download_file():
    """Attempt to connect to FTP and download the specified file."""
    try:
        # Connect to the FTP server
        ftp = ftplib.FTP(ftp_server)
        ftp.set_debuglevel(2)  # Enable FTP command logging
        ftp.set_pasv(True)  # Enable passive mode

        ftp.login(user=ftp_user, passwd=ftp_password)
        ftp.cwd(ftp_directory)

        # List files in the directory
        files = ftp.nlst()
        print("Files in the directory:")
        for file in files:
            print(file)

        # Download only the specified file
        if file_to_download in files:
            local_filename = os.path.join(local_directory, file_to_download)  # Save in the folder
            with open(local_filename, 'wb') as local_file:
                ftp.retrbinary('RETR ' + file_to_download, local_file.write)
                print(f"Downloaded {file_to_download} to {local_directory}")
        else:
            print(f"{file_to_download} not found in the directory.")
        
        ftp.quit()  # Ensure connection is closed

    except ftplib.all_errors as e:
        print(f"FTP error: {e}")
        raise  # Raise the error to be handled by the retry mechanism


# Retry mechanism
for attempt in range(max_retries):
    try:
        download_file()  # Call the function to perform the FTP download
        break  # If the download succeeds, exit the retry loop
    except (ftplib.error_temp, ConnectionResetError) as e:
        print(f"Temporary error occurred: {e}. Retrying ({attempt + 1}/{max_retries})...")
        if attempt == max_retries - 1:
            print("Max retries reached. Exiting.")
            raise e  # Re-raise the exception if all retries fail
        time.sleep(retry_delay)  # Wait before retrying


Files in the directory:
Fanatics-Product-Catalog_CUSTOM.txt.gz
Fanatics-Product-Catalog_IR.txt.gz
Fanatics-Top-Products_CUSTOM.txt.gz
Fanatics-Top-Products_IR.txt.gz
Fanatics-UK_CUSTOM.txt.gz
Fanatics-UK_IR.txt.gz
Fanatics-France_CUSTOM.txt.gz
Fanatics-France_IR.txt.gz
Fanatics-Spain_CUSTOM.txt.gz
Fanatics-Spain_IR.txt.gz
Fanatics-Germany_CUSTOM.txt.gz
Fanatics-Germany_IR.txt.gz
Fanatics-Australia_CUSTOM.txt.gz
Fanatics-Australia_IR.txt.gz
Fanatics-Italy_CUSTOM.txt.gz
Fanatics-Italy_IR.txt.gz
Downloaded Fanatics-Product-Catalog_IR.txt.gz to data-update-process


'221 Goodbye.'

### Decompress downloaded file

In [8]:
import gzip
import shutil
import os

# Directory containing the downloaded .txt.gz files
directory = os.path.join(os.getcwd(), "data-update-process")  # Path to the downloaded files

# Decompress each .txt.gz file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt.gz'):
        local_gz_filename = os.path.join(directory, filename)
        local_txt_filename = os.path.join(directory, filename[:-3])  # Remove the .gz extension

        # Decompress the .txt.gz file to .txt
        with gzip.open(local_gz_filename, 'rb') as f_in:
            with open(local_txt_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                print(f"Decompressed {local_gz_filename} to {local_txt_filename}")


Decompressed c:\Dynamic data\FDN\FDN-ProductHadnling\data-update-process\Fanatics-Product-Catalog_IR.txt.gz to c:\Dynamic data\FDN\FDN-ProductHadnling\data-update-process\Fanatics-Product-Catalog_IR.txt


### Convert to a Pandas DataFrame

In [3]:
import pandas as pd


# Get the current working directory
directory = os.path.join(os.getcwd(), "data-update-process")

# Path to the decompressed .txt file
local_txt_filename = os.path.join(directory, "Fanatics-Product-Catalog_IR.txt")

# Load the decompressed .txt file into a Pandas DataFrame
if os.path.exists(local_txt_filename):
    # Assuming the file is tab-delimited, adjust the delimiter if necessary
    df = pd.read_csv(local_txt_filename, delimiter='\t', low_memory=False)
    print(df.head())  # Display the first 5 rows
else:
    print(f"{local_txt_filename} does not exist.")

C:\Users\acer\AppData\Local\Temp\ipykernel_22360\4281405362.py:13: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_txt_filename, delimiter='\t')


   Unique Merchant SKU                                       Product Name  \
0              3180765  Men's New Era Black Minnesota Vikings Black On...   
1              5257010  Men's Under Armour Gray Virginia Tech Hokies S...   
2            200180351  Women's Antigua  Navy Jacksonville Jumbo Shrim...   
3            200465364   Keyscaper  Rey Mysterio Galaxy Impact Clear Case   
4            201060326  Women's Cutter & Buck  Gray Los Angeles Dodger...   

                                         Product URL  \
0  https://fanatics.93n6tx.net/c/5567077/669352/9...   
1  https://fanatics.93n6tx.net/c/5567077/669352/9...   
2  https://fanatics.93n6tx.net/c/5567077/669352/9...   
3  https://fanatics.93n6tx.net/c/5567077/669352/9...   
4  https://fanatics.93n6tx.net/c/5567077/669352/9...   

                                           Image URL  Current Price  \
0  https://feeds.frgimages.com/_pi3180000_ff_3180...          34.99   
1  https://feeds.frgimages.com/_pi5257000_ff_5257...      

### Filter needed products

In [4]:
df= df[df["Category"] == "NFL"]
df=df.head(1000)

In [5]:
df.shape

(1000, 70)

### Transformations to fit Shopify product import

In [6]:

# Count the occurrences of each value in the 'Text3' column of the 'nf' dataframe
# text3_counts = nf['Text3'].value_counts()

# Print the top 60 values
#  print(text3_counts.head(60))

nf = df.loc[:, ['Unique Merchant SKU', 'Product Name', 'Product URL', 'Image URL', 'Current Price', 'Original Price', 'Product Description', 'Gender', 'Text1', 'Text2', 'Text3','Size']]

rename_dict = {
    'Unique Merchant SKU': 'ID',
    'Product Name': 'Title',
    'Product URL': 'Metafield: custom.product_url [url]',
    'Image URL': 'Image Src',
    'Current Price': 'Variant Price',
    'Original Price': 'Variant Compare At Price',
    'Product Description': 'Body HTML',
    'Gender': 'Metafield: custom.product_gender [single_line_text_field]',
    'Text1': 'Metafield: custom.text1 [single_line_text_field]',
    'Text2': 'Metafield: custom.text2 [single_line_text_field]',
    'Text3': 'Tags',
    'Size': 'Metafield: custom.sizes [single_line_text_field]'
}

nf = nf.rename(columns=rename_dict)



## remove emptyspaces in Tags column between values
nf['Tags'] = nf['Tags'].str.replace(' ', '')

# Create an additional column 'Variant Inventory Tracker' with all values value 'shopify'
nf['Variant Inventory Tracker'] = 'shopify'
nf['Variant Inventory Policy'] = 'continue'
nf['Variant Fulfillment Service'] = 'manual'
nf['Variant Inventory Qty'] = '1'


### Save to google sheet

In [7]:
#./myenv/Scripts/Activate.ps1
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

from gspread_dataframe import set_with_dataframe

# Define the scope
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name(process.env.CREDENTIALS_GOOGLE_CLOUD, scope)

# Authorize the clientsheet
client = gspread.authorize(creds)

# Open the Google Sheet (by name or by key)
spreadsheet = client.open("Fanatics_product_import")

# Select the first sheet (or specify another sheet)
sheet = spreadsheet.sheet1  


# Clear existing data in the sheet (optional)
sheet.clear()

# Write DataFrame to Google Sheet
set_with_dataframe(sheet, nf)
